In [35]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import Callback 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

In [25]:
classifier = Sequential()

In [26]:
# Add convolution layer
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu'))
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), activation='relu'))

In [27]:
# Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [28]:
# More convolution
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), activation='relu'))
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), activation='relu'))

In [29]:
# Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [30]:
# Flatten
classifier.add(Flatten())

In [31]:
# Add full connection
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate =0.1))
classifier.add(Dense(units=46, activation='softmax'))

In [32]:
# Compiling the ANN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
__________

In [34]:
plot_model(classifier, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(
        '/Users/aryaprabhudesai/Desktop/PPL/Train',
        color_mode='grayscale',
        target_size=(28,28),
        batch_size=32,
        class_mode='categorical')

In [ ]:
test_set = test_datagen.flow_from_directory(
        '/Users/aryaprabhudesai/Desktop/PPL/Test',
        color_mode='grayscale',
        target_size=(28,28),
        batch_size=32,
        class_mode='categorical')

In [ ]:
import os
if not os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
del os

In [ ]:
checkpoint_path = 'checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.h5'
model_checkpoint = ModelCheckpoint(
        checkpoint_path, monitor='loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss')
callbacks_list = [model_checkpoint, early_stopping]

In [ ]:
# train model
history = classifier.fit_generator(
        train_set,
        steps_per_epoch=train_set.n//train_set.batch_size,
        epochs= 10,
        validation_data=test_set,
        validation_steps=test_set.n//test_set.batch_size)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
classifier.save('model_keras1_new.h5')

In [ ]:
def validate(classifier, test_set, steps):
    import numpy as np
    correct = 0
    n_guesses = 0
    for i in range(steps):
        a = test_set.next()
        prediction = a[0]
        yhat = classifier.predict(prediction)
        y = a[1]
        for i in range(len(yhat)):
            n_guesses += 1
            if np.argmax(yhat[i]) == np.argmax(y[i]):
                correct += 1
    return float(correct)/float(n_guesses)

In [ ]:
print(validate(classifier, test_set, 10))